In [ ]:
pickle

In [ ]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

In [ ]:
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]

    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched
    x_span = x_start_stop[1] - x_start_stop[0]
    y_span = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    x_step_pix = np.int(xy_window[0]*(1-xy_overlap[0]))
    y_step_pix = np.int(xy_window[1]*(1-xy_overlap[1]))
    # Compute the number of windows in x/y
    #x_nwin = np.int(x_span/xy_window[0]*1/xy_overlap[0]-(1/xy_overlap[0]-1))
    #y_nwin = np.int(y_span/xy_window[1]*1/xy_overlap[1]-(1/xy_overlap[1]-1))
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    x_nwin = np.int((x_span-nx_buffer)/x_step_pix) 
    y_nwin = np.int((y_span-ny_buffer)/y_step_pix)
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    for y_val in range(y_nwin):
        for x_val in range(x_nwin):
            # Calculate each window position
            x_start = x_val * x_step_pix + x_start_stop[0]
            y_start = y_val * y_step_pix + y_start_stop[0]
            x_end = x_start + xy_window[0]
            y_end = y_start +xy_window[1]
            # Append window position to list
            window_list.append(((x_start,y_start),(x_end,y_end)))
    # Return the list of windows
    return window_list

In [ ]:
def single_image_features(img, cspace='RGB', orient=9, pix_per_cell=8, cell_per_block=2, hog_channel=0, hist = True, spat = True):
    # Create a list to append feature vectors to
    img_features = []
    # apply color conversion if other than 'RGB'
    if cspace != 'RGB':
        if cspace == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif cspace == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif cspace == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif cspace == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif cspace == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        elif cspace == 'gray':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else: feature_image = np.copy(img)       

    if hog_channel == 'ALL':
        hog_features = []
        for channel in range(feature_image.shape[2]):
            hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                orient, pix_per_cell, cell_per_block, 
                                vis=False, feature_vec=True))
        hog_features = np.ravel(hog_features)

    else:
        hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                    pix_per_cell, cell_per_block, vis=False, feature_vec=True)
    img_features.append(hog_features)
    
    if spat == True:
        spat_size = (32,32)
        spat_features = bin_spatial(feature_image, size=spat_size)
        img_features.append(spat_features)
    if hist == True:
        hist_bins = 32
        hist_range = (0,256)
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
        img_features.append(hist_features)
    # , hist_features, spat_features
    # Append the new feature vector to the features list

    img_features=np.concatenate((img_features))

    # Return list of feature vectors
    return img_features

In [ ]:
# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', orient=9, 
                    pix_per_cell=8, cell_per_block=2, hog_channel=0):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))

        #4) Extract features for that window using single_img_features()
        features = single_image_features(test_img, cspace=color_space, orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, hog_channel=hog_channel, hist = True, spat = False)
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows

In [ ]:
test_images_windows = []
hot_windows_set = []
for image in test_images:
    window_img = np.copy(image)
    for win_size in [64,128,192,256,320]:
        windows = slide_window(image, x_start_stop=[None, None], y_start_stop=[np.int(image.shape[0]/2), image.shape[0]-70], 
                        xy_window=(win_size, win_size), xy_overlap=(0.2, 0.2))

        hot_windows = search_windows(image, windows, svc, feature_scaler, color_space=colorspace, orient=orient, 
                                 pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel)

        window_img = draw_boxes(window_img, hot_windows, color=(0, 0, 255), thick=6)
    
    plt.figure()
    plt.imshow(window_img)
    hot_windows_set.append(hot_windows)

In [ ]:
def video_own(image):
    window_img = np.copy(image)
    for win_size in [64,128,192,256,320]:
        windows = slide_window(image, x_start_stop=[None, None], y_start_stop=[np.int(image.shape[0]/2), image.shape[0]-70], 
                        xy_window=(win_size, win_size), xy_overlap=(0.2, 0.2))

        hot_windows = search_windows(image, windows, svc, feature_scaler, color_space=colorspace, orient=orient, 
                                 pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel)

        window_img = draw_boxes(window_img, hot_windows, color=(0, 0, 255), thick=6)
    return window_img

In [ ]:
from moviepy.editor import VideoFileClip

In [ ]:
output = 'test_video_output_own.mp4'
clip1 = VideoFileClip("test_video.mp4")

road_clip = clip1.fl_image(video_own) #NOTE: this function expects color images!!
%time road_clip.write_videofile(output, audio=False)